<a href="https://colab.research.google.com/github/Nagajahnavikancharla/HDS523007Homework/blob/main/Week05%5CDask_week5_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Name: Naga Jahnavi Kancharla
Assignment: Week 5
Course number: Hds5230-07

Importing required libraries

In [ ]:
import dask
import dask.dataframe as dd
import pandas as pd
import numpy as np
from datetime import datetime

In [ ]:

!pip install "dask[distributed]"






#defining datatypes

In [ ]:
dtypes = {
    'population': 'float64',
    'deaths': 'float64',
    'cases': 'float64',
    'state': 'object',
    'country': 'object',
    'level': 'object',
    'date': 'object',
    'aggregate': 'object',
    'city': 'object'
}
# Reading the CSV file and loading it into a dask dataframe.
covid_data = dd.read_csv('/content/timeseries.csv', dtype=dtypes)

#Creating a new dataframe object that consists of samples  corresponding to states in the US.

us_states_dd = covid_data[covid_data['country'] == 'United States']
us_states_dd = us_states_dd[us_states_dd['level'] == 'state']

The above code filters the DataFrame to include only rows where the country is 'United States' and where the level is 'state', with the operation performed in parallel by Dask, dividing the data into segments and applying the filters concurrently to each segment.

In [ ]:
#3.2
us_states_dd['date'] = dd.to_datetime(us_states_dd['date'])

mask = (us_states_dd['date'] >= '2020-01-01') & (us_states_dd['date'] <= '2021-02-28')
states_filtered = us_states_dd[mask]

In [ ]:
state_metrics = states_filtered.groupby('state').agg({
    'deaths': 'max',
    'population': 'mean'
}).compute()

state_metrics['per_capita_mortality'] = state_metrics['deaths'] / state_metrics['population']

# Ranking  states by per-capita mortality
mortality_ranking = state_metrics.sort_values('per_capita_mortality', ascending=False)

print("\nTop 10 states by per-capita mortality:")
print(mortality_ranking[['deaths', 'population', 'per_capita_mortality']].head(10))

/usr/local/lib/python3.11/dist-packages/dask/dataframe/io/csv.py:195: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = reader(bio, **kwargs)



Top 10 states by per-capita mortality:
                   deaths  population  per_capita_mortality
state                                                      
New Jersey        15211.0   8882190.0              0.001713
New York          24904.0  19453561.0              0.001280
Connecticut        4335.0   3565287.0              0.001216
Massachusetts      8183.0   6892503.0              0.001187
Rhode Island        960.0   1059361.0              0.000906
Washington, D.C.    559.0    705749.0              0.000792
Louisiana          3288.0   4648794.0              0.000707
Michigan           6218.0   9986857.0              0.000623
Illinois           7020.0  12671821.0              0.000554
Maryland           3243.0   6045680.0              0.000536


For this code, I have used different functions like filtering, grouping, and aggregating. I think parallelism makes sense because each partition of the dataset can be processed independently before combining the results.

From the results we can see that Newjersy has the highest moratality rate

In [ ]:
#3.3
states_filtered['year_month'] = states_filtered['date'].dt.to_period('M')

monthly_metrics = states_filtered.groupby(['state', 'year_month']).agg({
    'deaths': 'max',
    'cases': 'max'
}).compute()



monthly_metrics['cfr'] = monthly_metrics['deaths'] / monthly_metrics['cases']

# Reshaping  to get the matrix
cfr_matrix = monthly_metrics['cfr'].unstack()
print(cfr_matrix)
print(cfr_matrix.shape)

year_month                     2020-03   2020-04   2020-05   2020-06  \
state                                                                  
Virginia                      0.009608  0.034835  0.030824  0.028098   
Washington                    0.039828  0.056930  0.049356  0.038592   
Alabama                       0.013013  0.038325  0.035094  0.024970   
Alaska                             NaN  0.025352  0.021368  0.010705   
Arizona                            NaN  0.041841  0.045295  0.014934   
Arkansas                      0.014184  0.018740  0.018825  0.013462   
California                    0.021279  0.040429  0.037282  0.026362   
Colorado                      0.024321  0.051904  0.054810  0.051685   
Connecticut                   0.018004  0.081480  0.093856  0.093328   
Delaware                      0.031348  0.032108  0.038742  0.044509   
Washington, D.C.              0.018182  0.051816  0.052949  0.053355   
Florida                       0.012609  0.037637  0.043695  0.02

I think distributed computing is beneficial when working with large datasets. Since this is a moderate-sized dataset, parallel computing makes more sense. Although distributed computing yields results faster compared to parallel computing, parallel computing is sufficient in this case."


In [ ]:
#3.4
from dask.distributed import Client # Import the Client class from dask.distributed

client = Client()

cfr_changes = cfr_matrix.diff(axis=1)

# Aggregating changes
total_change = cfr_changes.abs().sum(axis=1)

# Ranking states by total CFR change
cfr_change_ranking = total_change.sort_values(ascending=False)



print("\nTop 10 states by total CFR change:")
print(cfr_change_ranking.head(10))

/usr/local/lib/python3.11/dist-packages/distributed/node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 34939 instead
  warnings.warn(
INFO:distributed.scheduler:State start
INFO:distributed.scheduler:  Scheduler at:     tcp://127.0.0.1:40481
INFO:distributed.scheduler:  dashboard at:  http://127.0.0.1:34939/status
INFO:distributed.scheduler:Registering Worker plugin shuffle
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:43511'
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:42285'
INFO:distributed.scheduler:Register worker <WorkerState 'tcp://127.0.0.1:36521', name: 0, status: init, memory: 0, processing: 0>
INFO:distributed.scheduler:Starting worker compute stream, tcp://127.0.0.1:36521
INFO:distributed.core:Starting established connection to tcp://127.0.0.1:60884
INFO:distributed.scheduler:Register worker <WorkerState 'tcp://127.0.0.1:36971', name: 1, status: init, memory:


Top 10 states by total CFR change:
state
Northern Mariana Islands        0.078341
Connecticut                     0.076915
Michigan                        0.072161
New Jersey                      0.069866
Missouri                        0.064549
Pennsylvania                    0.062147
Massachusetts                   0.061106
New York                        0.060628
United States Virgin Islands    0.059253
Wisconsin                       0.057251
dtype: float64


I used distributed computing for this code. This code was executed faster when compared to previous codes. But this is not a very large dataset, parallel computing makes sense .

Northern Mariana islands has the highest CFR change rate.